In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import re
import gensim
from gensim.models.word2vec import Word2Vec

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
data = pd.read_csv('/content/tweets.csv')

In [ ]:
pd.set_option('display.max_colwidth' , None)

In [ ]:
data

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile #sony #music #headphones https://instagram.com/p/5spiNsJ_c9/
7916,7917,0,"We would like to wish you an amazing day! Make every minute count #tls #today #iphone #accessories #news #life February 23, 2017 at 0…"
7917,7918,0,Helping my lovely 90 year old neighbor with her iPad this morning has just made me realise that 'I' don't actually need an I pad!
7918,7919,0,"Finally got my #smart #pocket #wifi stay connected anytime,anywhere! #ipad and #samsung #s3 #gadget # http://instagr.am/p/U-53G_vJU8/"


In [ ]:
data.drop('id',axis=1,inplace=True)

In [ ]:
data.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [ ]:
data.isna().sum()

label    0
tweet    0
dtype: int64

In [ ]:
data['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [ ]:
data.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [ ]:
#url removing

In [ ]:
def remove_URL(sample):
  return re.sub(r"http\S+", "", sample)

In [ ]:
data['text_clean_url-less'] = data['tweet'].apply(lambda x: remove_URL(x))

In [ ]:
#simple preprocessing using gensim

In [ ]:
data['clean_tweet'] = data['text_clean_url-less'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [ ]:
data.head()

,label,tweet,text_clean_url-less,clean_tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,#fingerprint #Pregnancy Test #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias]"
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect...,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect]"
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home]"
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [ ]:
data.drop(['tweet','text_clean_url-less'],axis = 1 ,inplace=True)

In [ ]:
data.head()

,label,clean_tweet
0,0,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,0,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias]"
2,0,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect]"
3,0,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home]"
4,1,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [ ]:
data.shape

(7920, 2)

In [ ]:
#splitting data into train data and test data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data['clean_tweet'],data['label'],test_size=0.2,random_state=42)

In [ ]:
#word embedding with word2vec

In [ ]:
model_gs = Word2Vec(x_train, min_count = 1)

In [ ]:
model_gs.wv.most_similar('apple')

[('than', 0.9995522499084473),
 ('again', 0.9995375275611877),
 ('mac', 0.9995023012161255),
 ('store', 0.9994899034500122),
 ('from', 0.9994856119155884),
 ('more', 0.9994782209396362),
 ('its', 0.9993264675140381),
 ('work', 0.9993249773979187),
 ('fuckyou', 0.9992970824241638),
 ('battery', 0.9991795420646667)]

In [ ]:
words = model_gs.wv.index_to_key

In [ ]:
len(words)

14946

In [ ]:
x_train_vec = np.array([np.array([model_gs.wv[i] for i in ls if i in words]) for ls in x_train],dtype=object)
x_test_vec = np.array([np.array([model_gs.wv[i] for i in ls if i in words]) for ls in x_test],dtype=object)

In [ ]:
len(x_train_vec[0][0])


100

In [ ]:
x_train_vec.ndim

1

In [ ]:
x_train_vec_avg = []
for v in x_train_vec:
  if v.size:
    x_train_vec_avg.append(v.mean(axis = 0))
  else:
    x_train_vec_avg.append(np.zeros(100,dtype=float))
x_test_vec_avg = []
for v in x_test_vec:
  if v.size:
    x_test_vec_avg.append(v.mean(axis = 0))
  else:
    x_test_vec_avg.append(np.zeros(100,dtype=float))

Trying out different models

In [ ]:
#Random Forest Classifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_train_vec_avg,y_train)

RandomForestClassifier()

In [ ]:
#training accuracy
y_pred = clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.9995265151515151

In [ ]:
#testing

In [ ]:
y_pred = clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8428030303030303

In [ ]:
#SVC

In [ ]:
svc = SVC(kernel='rbf')

In [ ]:
svc.fit(x_train_vec_avg,y_train)

SVC()

In [ ]:
#training accuracy svm
y_pred = svc.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8424873737373737

In [ ]:
#testing svc

In [ ]:
y_pred = svc.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8453282828282829

In [ ]:
#XGB

In [ ]:
xgb_clf = xgb.XGBClassifier(n_estimators=20 ,max_depth = 1, random_state = 42, learning_rate = 0.4, gamma = 0.1 )

In [ ]:
xgb_clf.fit(x_train_vec_avg,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
#training accuracy xgb
y_pred = xgb_clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8428030303030303

In [ ]:
#testing xgb

In [ ]:
y_pred = xgb_clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8453282828282829

In [ ]:
#KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25, metric = 'minkowski',p = 1)

In [ ]:
knn.fit(x_train_vec_avg,y_train)

KNeighborsClassifier(n_neighbors=25, p=1)

In [ ]:
#training accuracy knn
y_pred = knn.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8516414141414141

In [ ]:
y_pred = knn.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8402777777777778

In [ ]:
#Neural network

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(70,activation = 'relu'))
model.add(Dense(70,activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
x_train_new = np.array(x_train_vec_avg)
y_train_new = np.array(y_train)

In [ ]:
history = model.fit(x_train_new,y_train_new,epochs=100)

Epoch 1/100
198/198 [==============================] - 1s 2ms/step - loss: 0.4015 - accuracy: 0.8079
Epoch 2/100
198/198 [==============================] - 0s 1ms/step - loss: 0.3539 - accuracy: 0.8343
Epoch 3/100
198/198 [==============================] - 0s 2ms/step - loss: 0.3539 - accuracy: 0.8370
Epoch 4/100
198/198 [==============================] - 0s 1ms/step - loss: 0.3471 - accuracy: 0.8398
Epoch 5/100
198/198 [==============================] - 0s 2ms/step - loss: 0.3506 - accuracy: 0.8368
Epoch 6/100
198/198 [==============================] - 0s 1ms/step - loss: 0.3444 - accuracy: 0.8396
Epoch 7/100
198/198 [==============================] - 0s 1ms/step - loss: 0.3421 - accuracy: 0.8419
Epoch 8/100
198/198 [==============================] - 0s 2ms/step - loss: 0.3445 - accuracy: 0.8425
Epoch 9/100
198/198 [==============================] - 0s 1ms/step - loss: 0.3448 - accuracy: 0.8411
Epoch 10/100
198/198 [==============================] - 0s 2ms/step - loss: 0.3408 - accura

In [ ]:
x_test_new = np.array(x_test_vec_avg)
y_test_new = np.array(y_test)

In [ ]:
#testing

In [ ]:
model.evaluate(x_test_new,y_test_new)

50/50 [==============================] - 0s 1ms/step - loss: 0.3330 - accuracy: 0.8516


[0.33299902081489563, 0.8516414165496826]

Picking the better performing model

In [ ]:
#Neural network was the better performing model

In [ ]:
#random tweet test

In [ ]:
label_dict = {1 :'Negative sentiment on the product',0 :'Positive sentiment on the product'}

In [ ]:
def sentiment_analyze(tweet):
  tweet = remove_URL(tweet)
  tweet1 = gensim.utils.simple_preprocess(tweet)
  tweet_vec_avg = np.array([model_gs.wv[i] for i in tweet1 if i in words]).mean(axis = 0)
  test = tweet_vec_avg.reshape(1,100)
  sentiment = model.predict(test)
  sentiment[sentiment<0.5] = 0
  sentiment[sentiment>0.5] = 1
  print(label_dict[sentiment[0][0]])

In [ ]:
tweet = 'wow!!! my apple iphone 11 is simply the best #iphonelife #love #awesome'

In [ ]:
sentiment_analyze(tweet)

1/1 [==============================] - 0s 18ms/step
Positive sentiment on the product


In [ ]:
tweet2 = 'wow!! apple devices have become a nightmare.stupid phone'

In [ ]:
sentiment_analyze(tweet2)

1/1 [==============================] - 0s 15ms/step
Negative sentiment on the product
